In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency

# Loading the dataset
healthcare_data = pd.read_csv("c_Healthcare_dataset.csv")
healthcare_data.head()


,Ptid,Persistency_Flag,Gender,Race,Ethnicity,Region,Age_Bucket,Ntm_Speciality,Ntm_Specialist_Flag,Ntm_Speciality_Bucket,...,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls,Count_Of_Risks
0,P1,Persistent,Male,Caucasian,Not Hispanic,West,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0
1,P2,Non-Persistent,Male,Asian,Not Hispanic,West,55-65,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,0
2,P3,Non-Persistent,Female,Other/Unknown,Hispanic,Midwest,65-75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,Y,N,N,N,N,N,N,N,2
3,P4,Non-Persistent,Female,Caucasian,Not Hispanic,Midwest,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,1
4,P5,Non-Persistent,Female,Caucasian,Not Hispanic,Midwest,>75,GENERAL PRACTITIONER,Others,OB/GYN/Others/PCP/Unknown,...,N,N,N,N,N,N,N,N,N,1


In [2]:
# Check data types and structure
print("Data types of each column:")
print(healthcare_data.dtypes)

# Get summary statistics for numeric columns
print("\nSummary statistics for numeric columns:")
print(healthcare_data.describe())

# Check unique values for categorical columns
categorical_columns = healthcare_data.select_dtypes(include=['object']).columns
print("\nCategorical columns and their unique values:")
for col in categorical_columns:
    print(f"{col}: {healthcare_data[col].nunique()} unique values")


Data types of each column:
Ptid                              object
Persistency_Flag                  object
Gender                            object
Race                              object
Ethnicity                         object
                                   ...  
Risk_Hysterectomy_Oophorectomy    object
Risk_Estrogen_Deficiency          object
Risk_Immobilization               object
Risk_Recurring_Falls              object
Count_Of_Risks                     int64
Length: 69, dtype: object

Summary statistics for numeric columns:
       Dexa_Freq_During_Rx  Count_Of_Risks
count          3424.000000     3424.000000
mean              3.016063        1.239486
std               8.136545        1.094914
min               0.000000        0.000000
25%               0.000000        0.000000
50%               0.000000        1.000000
75%               3.000000        2.000000
max             146.000000        7.000000

Categorical columns and their unique values:
Ptid: 3424 unique valu

In [3]:
# Check for missing values in each column
missing_values = healthcare_data.isnull().sum()
print("\nNumber of missing values in each column:")
print(missing_values[missing_values > 0])



Number of missing values in each column:
Series([], dtype: int64)


In [4]:
numeric_data = healthcare_data.select_dtypes(include=['int64', 'float64'])
correlation_matrix = numeric_data.corr()
print(correlation_matrix)
print("The dataset contains very few numeric columns, and correlation between Dexa_Freq_During_Rx and Count_Of_Risks appears to be a very weak correlation (0.014). Thus, these do not show a significant linear relationship.") 

                     Dexa_Freq_During_Rx  Count_Of_Risks
Dexa_Freq_During_Rx             1.000000        0.013964
Count_Of_Risks                  0.013964        1.000000
The dataset contains very few numeric columns, and correlation between Dexa_Freq_During_Rx and Count_Of_Risks appears to be a very weak correlation (0.014). Thus, these do not show a significant linear relationship.


In [5]:
# discover categorical relationships

categorical_cols = ['Gender', 'Race', 'Ethnicity', 'Region', 'Persistency_Flag']
categorical_dist = {col: healthcare_data[col].value_counts() for col in categorical_cols}

for col, dist in categorical_dist.items():
    print(f"Distribution for {col}:")
    print(dist)

print("The datasets is mostly consists of female (3230) over male. Most entries are Caucasian (3148). The majority patients are Not Hispanic (3235). Most are from Midwest (1383) and South (1247).  There are more Non-Persistent (2135) and Persistent (1289).")



Distribution for Gender:
Gender
Female    3230
Male       194
Name: count, dtype: int64
Distribution for Race:
Race
Caucasian           3148
Other/Unknown         97
African American      95
Asian                 84
Name: count, dtype: int64
Distribution for Ethnicity:
Ethnicity
Not Hispanic    3235
Hispanic          98
Unknown           91
Name: count, dtype: int64
Distribution for Region:
Region
Midwest          1383
South            1247
West              502
Northeast         232
Other/Unknown      60
Name: count, dtype: int64
Distribution for Persistency_Flag:
Persistency_Flag
Non-Persistent    2135
Persistent        1289
Name: count, dtype: int64
The datasets is mostly consists of female (3230) over male. Most entries are Caucasian (3148). The majority patients are Not Hispanic (3235). Most are from Midwest (1383) and South (1247).  There are more Non-Persistent (2135) and Persistent (1289).


In [6]:
# Data cleaning
# Select only numeric columns from the dataset to avoid errors
numeric_data = healthcare_data.select_dtypes(include=['int64', 'float64'])

# Calculate Q1 (25th percentile) and Q3 (75th percentile) for numeric columns
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)

# Compute the Interquartile Range (IQR)
IQR = Q3 - Q1

# Detect outliers based on the IQR method
outliers = ((numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR))).sum()

# Print the number of outliers detected in each column
print("\nNumber of outliers detected in each column:")
print(outliers[outliers > 0])



Number of outliers detected in each column:
Dexa_Freq_During_Rx    460
Count_Of_Risks           8
dtype: int64


In [7]:
# Data cleaning
# Check for skewness in numeric columns
skewness = numeric_data.skew().sort_values(ascending=False)
print("\nSkewness of numeric columns:")
print(skewness)



Skewness of numeric columns:
Dexa_Freq_During_Rx    6.808730
Count_Of_Risks         0.879791
dtype: float64


In [8]:
# Capping outliers using the IQR method
for col in numeric_data.columns:
    Q1 = numeric_data[col].quantile(0.25)
    Q3 = numeric_data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    healthcare_data[col] = numeric_data[col].clip(lower=lower_bound, upper=upper_bound)


In [9]:
import numpy as np
# Calculate skewness for numeric columns
numeric_data = healthcare_data.select_dtypes(include=['int64', 'float64'])  # Only numeric columns
skewness = numeric_data.skew().sort_values(ascending=False)

#Apply log transformation to highly skewed columns (skewness > 1)
high_skew = skewness[skewness > 1].index  # Get columns with high skewness
healthcare_data[high_skew] = healthcare_data[high_skew].apply(lambda x: np.log1p(x))  # Apply log1p (log(x+1))

# Check which columns were transformed
print("\nLog transformation applied to highly skewed columns:")
print(high_skew)



Log transformation applied to highly skewed columns:
Index(['Dexa_Freq_During_Rx'], dtype='object')


In [10]:
# Remove duplicate rows
df = healthcare_data.drop_duplicates()

# Check if any duplicates remain
print("Number of duplicate rows:", df.duplicated().sum())


Number of duplicate rows: 0


In [11]:
# Drop irrelevant columns (e.g., an ID column that doesn't contribute to the analysis)
df = df.drop(['Ptid'], axis=1)

# Check for columns with a single unique value and drop them
for col in df.columns:
    if df[col].nunique() == 1:
        print(f"Dropping column {col} as it contains only one unique value.")
        df = df.drop(col, axis=1)


In [12]:
# List all the columns to identify the correct text column
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

print(df.columns)


Index(['Persistency_Flag', 'Gender', 'Race', 'Ethnicity', 'Region',
       'Age_Bucket', 'Ntm_Speciality', 'Ntm_Specialist_Flag',
       'Ntm_Speciality_Bucket', 'Gluco_Record_Prior_Ntm',
       'Gluco_Record_During_Rx', 'Dexa_Freq_During_Rx', 'Dexa_During_Rx',
       'Frag_Frac_Prior_Ntm', 'Frag_Frac_During_Rx', 'Risk_Segment_Prior_Ntm',
       'Tscore_Bucket_Prior_Ntm', 'Risk_Segment_During_Rx',
       'Tscore_Bucket_During_Rx', 'Change_T_Score', 'Change_Risk_Segment',
       'Adherent_Flag', 'Idn_Indicator', 'Injectable_Experience_During_Rx',
       'Comorb_Encounter_For_Screening_For_Malignant_Neoplasms',
       'Comorb_Encounter_For_Immunization',
       'Comorb_Encntr_For_General_Exam_W_O_Complaint,_Susp_Or_Reprtd_Dx',
       'Comorb_Vitamin_D_Deficiency',
       'Comorb_Other_Joint_Disorder_Not_Elsewhere_Classified',
       'Comorb_Encntr_For_Oth_Sp_Exam_W_O_Complaint_Suspected_Or_Reprtd_Dx',
       'Comorb_Long_Term_Current_Drug_Therapy', 'Comorb_Dorsalgia',
       'Comorb_Pers

In [13]:
cross_tab_presistency_gender = pd.crosstab(healthcare_data['Gender'], healthcare_data['Persistency_Flag'])
print("Cross-tabulation between Gender and Peristency Flag:")
print(cross_tab_presistency_gender)
print("The Cross-tabulation between Gender and Peristency show that the higher of Non-persistency and Persistency are female. Smaller portion of males fall into either category.")

Cross-tabulation between Gender and Peristency Flag:
Persistency_Flag  Non-Persistent  Persistent
Gender                                      
Female                      2018        1212
Male                         117          77
The Cross-tabulation between Gender and Peristency show that the higher of Non-persistency and Persistency are female. Smaller portion of males fall into either category.


In [14]:
cross_tab_race_persistency = pd.crosstab(healthcare_data['Race'],  healthcare_data['Persistency_Flag'])
print("Cross-tabulation between Race and Peristency Flag:")
print(cross_tab_race_persistency)
print("The Cross-tabulation between Race and Peristency show that the higher of Non-persistency and Persistency are Caucasian. Smaller portion of African American, Asian, and Other/Unknown are represented in both categories.")

Cross-tabulation between Race and Peristency Flag:
Persistency_Flag  Non-Persistent  Persistent
Race                                        
African American              65          30
Asian                         43          41
Caucasian                   1963        1185
Other/Unknown                 64          33
The Cross-tabulation between Race and Peristency show that the higher of Non-persistency and Persistency are Caucasian. Smaller portion of African American, Asian, and Other/Unknown are represented in both categories.


In [15]:
cross_tab_age_persistency = pd.crosstab(healthcare_data['Age_Bucket'],  healthcare_data['Persistency_Flag'])
print("Cross-tabulation between Age and Peristency Flag:")
print(cross_tab_age_persistency)
print("The Cross-tabulation between Age Buckcet and Peristency show that <55 has the smallest population, and the rest of the Age Buckest seem approximately normally distributed.")

Cross-tabulation between Age and Peristency Flag:
Persistency_Flag  Non-Persistent  Persistent
Age_Bucket                                  
55-65                        472         261
65-75                        653         433
<55                          103          63
>75                          907         532
The Cross-tabulation between Age Buckcet and Peristency show that <55 has the smallest population, and the rest of the Age Buckest seem approximately normally distributed.


In [16]:
cross_tab_region_persistency = pd.crosstab(healthcare_data['Region'],  healthcare_data['Persistency_Flag'])
print("Cross-tabulation between Region and Peristency Flag:")
print(cross_tab_region_persistency)
print("The Cross-tabulation between Region and Peristency shows that most patients are from Midwest (934) and South (753). And Other/Unknown (35) region is the least populated in the distribution.")

Cross-tabulation between Region and Peristency Flag:
Persistency_Flag  Non-Persistent  Persistent
Region                                      
Midwest                      934         449
Northeast                    134          98
Other/Unknown                 35          25
South                        753         494
West                         279         223
The Cross-tabulation between Region and Peristency shows that most patients are from Midwest (934) and South (753). And Other/Unknown (35) region is the least populated in the distribution.


In [17]:
from scipy.stats import chi2_contingency

# Perform chi-square tests fro Race, Age Bucket, Region individually vs. Persistency Flag. 

chi2_race, p_race, dof_race, expected_race = chi2_contingency(cross_tab_race_persistency)
print(f"Chi_square test for Race vs. Persistency: p-value: {p_race}")
print("There is no significant relationship between Race and Persistency_Flag, the p-value is greater than the 0.05 significance level.\n")

chi2_age, p_age, dof_age, expected_age = chi2_contingency(cross_tab_age_persistency)
print(f"Chi_square test for Age Bucket vs. Persistency: p-value: {p_age}")
print("There is no significant relationship between Age Bucket and Persistency_Flag, the p-value is greater than the 0.05 significance level.\n")


chi2_region, p_region, dof_region, expected_region = chi2_contingency(cross_tab_region_persistency)
print(f"Chi_square test for Region vs. Persistency: p-value: {p_region}")
print("**There is significant relationship between Region and Persistency_Flag, the p-value is less than the 0.05 significance level.\n")


Chi_square test for Race vs. Persistency: p-value: 0.08996072887774281
There is no significant relationship between Race and Persistency_Flag, the p-value is greater than the 0.05 significance level.

Chi_square test for Age Bucket vs. Persistency: p-value: 0.2752451478185551
There is no significant relationship between Age Bucket and Persistency_Flag, the p-value is greater than the 0.05 significance level.

Chi_square test for Region vs. Persistency: p-value: 4.475541847690054e-06
**There is significant relationship between Region and Persistency_Flag, the p-value is less than the 0.05 significance level.



In [18]:
# analyze all categorical combinations to find out sifnificant relationships
categorical_columns = healthcare_data.select_dtypes(include=['object']).columns
chi_square_results_all = {} # dictionary to store all chi-square results between all pairs

# loop each combination of columns 
for col1 in categorical_columns:
    chi_square_results_all[col1] = {}
    for col2 in categorical_columns:
        if col1 != col2:
            cross_tab = pd.crosstab(healthcare_data[col1], healthcare_data[col2])
            chi2, p, dof, expected = chi2_contingency(cross_tab)
            chi_square_results_all[col1][col2] = {
                "Chi1_Statistic": chi2,
                "P-value": p
            }
# loop combinations that has a p-value less than the significance level 0.05
chi_square_summary = {}
for col1, tests in chi_square_results_all.items():
    for col2, result in tests.items():
        if result["P-value"]<0.05:
            if col1 not in chi_square_summary:
                chi_square_summary[col1] = {}
            chi_square_summary[col1][col2] = result["P-value"]

chi_square_summary_df = pd.DataFrame(chi_square_summary)

print("Significant Chi-square Test Results (p < 0.05):")
chi_square_summary_df.head(66)

Significant Chi-square Test Results (p < 0.05):


,Persistency_Flag,Gender,Race,Ethnicity,Region,Age_Bucket,Ntm_Speciality,Ntm_Specialist_Flag,Ntm_Speciality_Bucket,Gluco_Record_Prior_Ntm,...,Risk_Chronic_Liver_Disease,Risk_Family_History_Of_Osteoporosis,Risk_Low_Calcium_Intake,Risk_Vitamin_D_Insufficiency,Risk_Poor_Health_Frailty,Risk_Excessive_Thinness,Risk_Hysterectomy_Oophorectomy,Risk_Estrogen_Deficiency,Risk_Immobilization,Risk_Recurring_Falls
Region,4.475542e-06,NaN,6.458565e-32,4.082793e-09,NaN,NaN,1.753288e-69,3.017009e-36,3.311919e-54,3.239854e-10,...,NaN,0.001500,1.878166e-24,7.997006e-17,NaN,NaN,NaN,NaN,NaN,NaN
Ntm_Speciality,3.016264e-22,8.889112e-16,NaN,8.005601e-09,1.753288e-69,7.405681e-11,NaN,0.000000e+00,0.000000e+00,5.776598e-08,...,NaN,0.000022,7.183221e-05,7.305800e-07,NaN,NaN,NaN,8.379319e-67,1.637860e-09,NaN
Ntm_Specialist_Flag,4.647875e-16,NaN,NaN,4.144748e-03,3.017009e-36,4.225282e-04,0.000000e+00,NaN,0.000000e+00,6.270628e-04,...,NaN,0.000003,1.201675e-04,2.066148e-03,NaN,0.000498,NaN,NaN,NaN,NaN
Ntm_Speciality_Bucket,2.830657e-24,NaN,NaN,1.179187e-04,3.311919e-54,NaN,0.000000e+00,0.000000e+00,NaN,2.714163e-12,...,NaN,0.000022,4.725305e-09,3.964497e-03,NaN,0.000199,NaN,NaN,NaN,NaN
Gluco_Record_During_Rx,2.415090e-35,NaN,4.294458e-04,NaN,2.926675e-09,NaN,4.214914e-08,5.230105e-06,8.352088e-12,2.318015e-106,...,NaN,NaN,NaN,1.260126e-03,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tscore_Bucket_Prior_Ntm,NaN,NaN,NaN,NaN,NaN,7.346165e-23,1.792331e-06,NaN,NaN,NaN,...,NaN,0.000002,NaN,1.570929e-04,NaN,0.000654,0.03221,NaN,NaN,NaN
Risk_Chronic_Liver_Disease,NaN,NaN,NaN,NaN,NaN,6.620287e-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Risk_Hysterectomy_Oophorectomy,NaN,NaN,NaN,NaN,NaN,1.170964e-02,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Risk_Recurring_Falls,NaN,NaN,NaN,NaN,NaN,2.811047e-03,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.014367,0.000006,NaN,NaN,NaN,NaN


In [23]:
# Assuming df_encoded is the cleaned and processed dataset after previous transformations.
# Filter the 'significant_data' to include only the columns you find important.

# Define significant_data as a subset of the full dataset
# Assuming your processed DataFrame is called 'df', update this part accordingly
significant_data = df[['persistency_flag', 'dexa_during_rx', 'comorb_long_term_current_drug_therapy', 
                       'comorb_encounter_for_screening_for_malignant_neoplasms', 
                       'comorb_gastro_esophageal_reflux_disease']]


# Now proceed with numeric column filtering and encoding for categorical data
# Filter numeric columns for correlation
significant_numeric_data = significant_data.select_dtypes(include=['int64', 'float64'])

# Handle categorical columns by encoding them (e.g., with label encoding)
from sklearn.preprocessing import LabelEncoder

# Initialize a label encoder
le = LabelEncoder()

# List of categorical columns to encode
categorical_columns = significant_data.select_dtypes(include=['object']).columns

# Encode categorical columns into numeric
for col in categorical_columns:
    significant_data[col] = le.fit_transform(significant_data[col])

# Combine the numeric and encoded categorical columns for correlation analysis
encoded_significant_data = significant_data

# Generate the heatmap for correlation of the combined dataset
plt.figure(figsize=(12, 10))
sns.heatmap(encoded_significant_data.corr(), annot=True, cmap='coolwarm', linewidths=0.5)
plt.title('Correlation Heatmap of Significant Columns')
plt.show()


KeyError: "None of [Index(['persistency_flag', 'dexa_during_rx',\n       'comorb_long_term_current_drug_therapy',\n       'comorb_encounter_for_screening_for_malignant_neoplasms',\n       'comorb_gastro_esophageal_reflux_disease'],\n      dtype='object')] are in the [columns]"

In [20]:
# Extract the correlations with 'Persistency_Flag'
persistency_correlations = correlation_matrix['Persistency_Flag'].abs().sort_values(ascending=False)

# Filter the most significant correlations with 'Persistency_Flag' (absolute correlation > 0.2)
significant_persistency_correlations = persistency_correlations[persistency_correlations > 0.2]

# Display the filtered correlations
print("Features strongly correlated with Persistency_Flag (absolute correlation > 0.2):")
print(significant_persistency_correlations)


KeyError: 'Persistency_Flag'

In [21]:
import seaborn as sns
# Plot the distribution of the target variable
plt.figure(figsize=(6, 4))
sns.countplot(data=df, x='persistency_flag')
plt.title('Distribution of Persistency Flag')
plt.xlabel('Persistency Flag')
plt.ylabel('Count')
plt.show()

# Check class distribution
persistency_counts = df['persistency_flag'].value_counts()
print("\nClass Distribution of Persistency Flag:")
print(persistency_counts)


ValueError: Could not interpret value `persistency_flag` for `x`. An entry with this name does not appear in `data`.

<Figure size 600x400 with 0 Axes>

In [ ]:
# Example interaction between 'gender' and 'age_bucket_numeric'
sns.catplot(data=df, x='gender', hue='persistency_flag', col='age_bucket', kind='count', col_wrap=2)
plt.show()